In [1]:
from   ctypes import *
from    scipy import sparse

import  numpy as np
import  random

In [2]:
# treat this as a library call and put it here

# in Linux:

#     precompile with bash script
#     then run ldconfig

#     if your computer has issues you may need to reconfgure the PATH used to find libraries

# <my_computer/Projects/Machine_Learning$>>> $ gcc -fPIC -shared -o keith_functions.so keith_module.c
# <my_computer/Projects/Machine_Learning$>>> $ sudo ldconfig -v

# note that printf
# when called from keith_mutate over in C land
# prints out to the terminal you are using to run your IDE, not to the notebook here



In [3]:
keith_functions = CDLL("keith_functions.so")

In [4]:
keith_mutate    = keith_functions.keith_mutate

In [5]:
'''# speed comparison of four approaches    '''
'''    # base_mutate native python libraries        '''
'''    # pygad_mutate specialist libraries and two for loops'''
'''    # vector_mutate special libraries and aggressive vectorisation'''
'''    # keith_mutate modular call from bespoke C function in local library '''
'''                                                                                 '''
'''# further optimisations maybe possible with the four approaches                          '''
'''# note that mutate request internally casts the numpy population into a list of lists            '''

'# note that mutate request internally casts the numpy population into a list of lists            '

In [6]:
class MutateRequest:
    def __init__(self, population):
        
        # fairer to start with the native list object and require modules to promote to array
        # than have "naive" modules demoting ndarrays back down to lists
        number_of_mutations = get_number_of_mutations()
        self.population     = population
        self.population     = [list(algo) for algo in population] # see explanation of choice of list not ndarray
        self.rows_algos     = len(population)
        self.columns_genes  = len(population[0])
        self.number_of_mutations = number_of_mutations

class TellObject:
    def __init__(self, population, base_mutants, keith_mutants, pygad_mutants, vector_mutants):
        self.population     = population
        self.base_mutants   = base_mutants
        self.keith_mutants  = keith_mutants        
        self.pygad_mutants  = pygad_mutants
        self.vector_mutants = vector_mutants
        
def get_random_mutation_min_val():
    return -2

def get_random_mutation_max_val():
    return 2
        
def tell(tell_object):
    print('\noriginal population\n', np.around(np.asarray(tell_object.population), decimals=4))
    print('\nmutation by base libraries')
    for i in range(len(tell_object.base_mutants)):
          print(np.around(np.asarray(tell_object.base_mutants[i]),decimals=4))
    print('\nmutation by aggressively vectorised scipy\n', np.around(np.asarray(tell_object.vector_mutants), decimals=4))
    print('\nmutation by pygad function\n', np.around(np.asarray(tell_object.pygad_mutants), decimals=4))
    print('\nmutation by bespoke C function\n', np.around(np.asarray(tell_object.keith_mutants), decimals=4))

def get_number_of_mutations():
    number_of_mutations = 1
    return number_of_mutations

def build_population():
    low             = -1
    high            =  1
    rows_algos      =  4
    columns_genes   =  20
    cohort_size     = (rows_algos, columns_genes) #  rows_algos, columns_genes
    population      = np.random.uniform(low, high, size=cohort_size)
    return population

In [7]:
def base_mutate(mutate_request):
    '''only uses naive base library - math, itertools, random etc are allowed'''
    base_mutants        = mutate_request.population
    number_of_mutations = mutate_request.number_of_mutations
    rows_algos          = mutate_request.rows_algos
    columns_genes       = mutate_request.columns_genes

    for algo in base_mutants:
        for mutation in range(number_of_mutations):
            random_place    = random.randrange(columns_genes)
            starting_gene   = algo[random_place]
            a_float         = random.random()
            algo.pop(random_place)
            mutant_gene     = starting_gene + a_float
            algo.insert(random_place, mutant_gene)
    return base_mutants

In [8]:
def keith_wrapper(mutate_request):
    
    rows_algos    = mutate_request.rows_algos
    columns_genes = mutate_request.columns_genes
    
    population      = np.asarray(mutate_request.population)
    flat_population = list(np.ravel(np.copy(population)))
    
    first_gene              = (c_double(flat_population[0]))
    min_mutation            = get_random_mutation_min_val()
    max_mutation            = get_random_mutation_max_val()
    length_flat_population  = len(flat_population)
    number_of_mutations     = get_number_of_mutations()
    
    CCompatiblePopulation   = c_double * length_flat_population
    print('\nCCompatiblePopulation\n', CCompatiblePopulation)
    
    C_compatible_population = CCompatiblePopulation(*flat_population)
    print('\nC_compatible_population\n',  C_compatible_population)

    p                       = pointer(C_compatible_population)
    print('\np\n', p)
    
    print('\np values\n', p.contents)
    
    print("flat population before\n", flat_population)
    
    keith_mutate(p, c_int(min_mutation), c_int(max_mutation), c_int(length_flat_population), c_int(number_of_mutations))
    
    print("flat_population after\n", flat_population)
    keith_mutants = np.asarray(flat_population)

    return keith_mutants

In [9]:
'''
# keith wrapper and mutate
'''

def test_keith_wrapper():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = keith_wrapper(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    
    
    #form
    #assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    #assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    #assert isinstance(mutants[0][0],float),       'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
#test_keith_wrapper()

In [10]:
def pygad_mutate(mutate_request):
    # 1. removed dependency on gene_type
    # 2. all modules for these time trials start with a list because they refer to the MutateRequest
    #    and the mutate reqeust offers a list as the most basic thing
    #    so if this module seems handicapped by e.g. the need to retype offspring as an array then
    #    people are welcome to change the interface, but it just passes that handicap elsewhere

    offspring                = np.asarray(mutate_request.population)
    num_genes                = mutate_request.columns_genes
    mutation_num_genes       = mutate_request.number_of_mutations
    random_mutation_min_val  = get_random_mutation_min_val()
    random_mutation_max_val  = get_random_mutation_max_val()
    
    for offspring_idx in range(offspring.shape[0]):
        mutation_indices = np.array(random.sample(range(0, num_genes), mutation_num_genes))
        for gene_idx in mutation_indices:
            random_value = np.random.uniform(low=random_mutation_min_val, 
                                            high=random_mutation_max_val, 
                                            size=1)
            offspring[offspring_idx, gene_idx] = offspring[offspring_idx, gene_idx] + random_value
    return offspring

In [11]:
def vector_mutate(mutate_request):

    population          = np.asarray(mutate_request.population)
    number_of_mutations = mutate_request.number_of_mutations
    all_loci            = mutate_request.rows_algos * mutate_request.columns_genes
    mutation_rate       = number_of_mutations/mutate_request.columns_genes
    
    mutation_matrix = sparse.random(population.shape[0], population.shape[1], density=mutation_rate)
    pre_mutants     = np.copy(population)
    mutants         = np.add(pre_mutants, mutation_matrix.todense())
    return mutants

In [12]:
if __name__ == '__main__':
    
    population          = build_population()
    mutate_request      = MutateRequest(population)
    
    base_mutants        =  base_mutate(mutate_request)
    keith_mutants       = keith_wrapper(mutate_request)
    pygad_mutants       = pygad_mutate(mutate_request)
    vector_mutants      = vector_mutate(mutate_request)
    
    that_which_to_tell = TellObject(population, base_mutants, keith_mutants, pygad_mutants, vector_mutants)
    tell(that_which_to_tell)
    


CCompatiblePopulation
 <class '__main__.c_double_Array_80'>

C_compatible_population

p

p values
flat population before
 [-0.3545592090542644, -0.5784781276623161, 0.4244849590966895, -0.06886889466263835, -0.7972548776961981, 0.9389580578172345, 0.5429008902335621, 0.6384298183202799, 0.14681036500806854, -0.8330621743548452, -0.668715932523533, 0.4834857144215663, -0.0453896659299573, -0.6288595002232034, 0.08807988943865852, -0.9404333767711397, 0.425978926037605, -0.7414957033373075, 0.8268672218764705, -0.879740362695397, 0.5017218510052579, 0.47489719755406434, -0.7342623197771954, -0.03524248637262395, -0.49942131619773655, -0.6069898952090329, 0.46951921580143985, -0.042984353342894854, 0.9642375155482765, -0.2915539237558542, 0.46846312291658854, -0.5560413196396652, 0.8436734865152427, 0.9732795757769401, -0.9817565617465762, -0.9519031919424056, 0.26012428640099117, -0.5666042946981393, -0.41294153365861663, 0.21730636828581518, -0.4885230745753213, 0.8092085322410472, 0.5

In [13]:
'''
# build population
'''

def test_build_population():
    population = build_population()
    assert isinstance(population,       np.ndarray),    'Error: should be array of arrays'
    assert isinstance(population[0],    np.ndarray),    'Error: should be an array'
    assert isinstance(population[0][0], np.float64),    'Error: should be a float'
    print('pass')

In [14]:
'''
# base mutate
'''    

def test_base_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = base_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be a list'
    assert isinstance(mutants[0],   list),  'Error: should be a list'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')

In [15]:
'''
# vector mutate
'''

def test_vector_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = vector_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],np.matrix), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')

In [16]:
'''
# pygad mutate
'''
    
def test_pygad_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = pygad_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    


In [17]:
'''def test_none_are_same():
    population          = build_population()
    mutate_request      = MutateRequest(population)
    
    number_of_mutations = get_number_of_mutations()
    
    base_mutants        =  base_mutate(mutate_request)
    keith_mutants       = keith_mutate(mutate_request)
    pygad_mutants       = pygad_mutate(mutate_request)
    vector_mutants      = vector_mutate(mutate_request)
    
    if number_of_mutations > 0:
    
        assert  np.asarray(base_mutants).all()     !=  keith_mutants.all(), 'Error: really unlikely, more likely object blurring'
        assert  np.asarray(base_mutants).all()     !=  pygad_mutants.all(), 'Error: really unlikely, more likely object blurring'
        assert  np.asarray(base_mutants).all()     != vector_mutants.all(), 'Error: really unlikely, more likely object blurring'
        assert keith_mutants.all()     !=  pygad_mutants.all(), 'Error: really unlikely, more likely object blurring'
        assert keith_mutants.all()     != vector_mutants.all(), 'Error: really unlikely, more likely object blurring'
        assert pygad_mutants.all()     != vector_mutants.all(), 'Error: really unlikely, more likely object blurring'
        
    if number_of_mutations == 0:
        
        assert  np.asarray(base_mutants).all()     ==  keith_mutants.all(), 'Error: should be same'
        assert  np.asarray(base_mutants).all()     ==  pygad_mutants.all(), 'Error: should be same'
        assert  np.asarray(base_mutants).all()     == vector_mutants.all(), 'Error: should be same'
        assert keith_mutants.all()     ==  pygad_mutants.all(), 'Error: should be same'
        assert keith_mutants.all()     == vector_mutants.all(), 'Error: should be same'
        assert pygad_mutants.all()     == vector_mutants.all(), 'Error: should be same'
'''

"def test_none_are_same():\n    population          = build_population()\n    mutate_request      = MutateRequest(population)\n    \n    number_of_mutations = get_number_of_mutations()\n    \n    base_mutants        =  base_mutate(mutate_request)\n    keith_mutants       = keith_mutate(mutate_request)\n    pygad_mutants       = pygad_mutate(mutate_request)\n    vector_mutants      = vector_mutate(mutate_request)\n    \n    if number_of_mutations > 0:\n    \n        assert  np.asarray(base_mutants).all()     !=  keith_mutants.all(), 'Error: really unlikely, more likely object blurring'\n        assert  np.asarray(base_mutants).all()     !=  pygad_mutants.all(), 'Error: really unlikely, more likely object blurring'\n        assert  np.asarray(base_mutants).all()     != vector_mutants.all(), 'Error: really unlikely, more likely object blurring'\n        assert keith_mutants.all()     !=  pygad_mutants.all(), 'Error: really unlikely, more likely object blurring'\n        assert keith_mutan

In [18]:
 
'''
# tests per se
'''

def tests():
    
    test_base_mutate()
    test_build_population()
    #test_keith_wrapper()
    test_pygad_mutate()
    test_vector_mutate()
    print('\nwell done passed all active tests - stick to TDD even where you are excited.\n')
    #test_none_are_same()
    
tests()

pass
pass
pass
pass

well done passed all active tests - stick to TDD even where you are excited.

